In [5]:
from sim_env import FixedSimEnv
from main import run_ehd, flag_departure_event, recommended_dep_t, ehd_message, td
from copy import deepcopy
import pandas as pd
import numpy as np
from data_handlers import GTFSHandler, AVLHandler, bpf, write_sim_data, ODXHandler
from itertools import product
from constants import *
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from utils import plot_situation
from objects import Trip
from main import run_di
from tqdm import tqdm
from interlining import *

while loop:   8%|▊         | 959/12000 [00:33<03:25, 53.82it/s]

In [6]:
env = FixedSimEnv()
unique_dates = env.link_times['date'].unique()
day = np.random.choice(unique_dates, replace=False)

next_obs, rew, done, info = env.reset(hist_date='2022-11-15')
print(env.hist_date)

2022-11-15


In [7]:
env, info, n_steps, upd_info = run_di(env, done, n_steps=0, debug_ehd=True)
n_steps, td(env.time)

while loop:   1%|          | 143/12000 [00:02<03:45, 52.65it/s]

----
Time 0 days 05:27:24
Expected: 0 days 05:27:24
Headways: 0 days 00:07:17 -- 0 days 00:20:36
Schedule: 0 days 05:30:00
0 days 05:27:24 < Departure < 0 days 05:35:00
Recommended 0 days 05:34:03 ---> 0 days 05:34:03


(143, Timedelta('0 days 05:27:24'))

In [14]:
upd_info

{}

In [4]:
env, info, n_steps, upd_info = run_di(env, done, n_steps=n_steps, debug_ehd=True)
n_steps, td(env.time)

while loop:   8%|▊         | 958/12000 [00:18<03:25, 53.82it/s]

KeyboardInterrupt: 

In [42]:
def find_next_trip(env, trip, curr_stop_idx):
    dep_t = None
    stop_idx = None
    stops = env.stops[trip.direction]
    for i in range(curr_stop_idx-1, -1, -1):
        stop = stops[i]
        # there should not be a case where there is no recording
        if stop.last['trip_id'][-1] != trip.id:
            dep_t = stop.last['arrival_time'][-1]
            stop_idx = deepcopy(i)
            break
    if dep_t is None:
        dep_t = env.next_terminal_departure(trip.schedule['departure_sec'].values[0],
                                                trip.direction, trip.stops[0])
        stop_idx = 0
    return dep_t, stop_idx

In [41]:
def get_next_headway(env, control_vehs, terminal=False,
                        expected_dep=0):
    control_veh = env.vehicles[control_vehs.index[0]]
    trip = control_veh.next_trips[0] if terminal else control_veh.curr_trip
    expected_dep = max(env.time, expected_dep)

    route = env.routes[control_veh.route_id]
    dep_t, stop_idx_from = find_next_trip(route, trip,control_veh.stop_idx)

    print(dep_t, stop_idx_from, env.time)

    line = env.lines[control_veh.route_id]
    travel_time = line.time_between_two_stops(stop_idx_from, control_veh.stop_idx, 
                                                dep_t, trip.stops)
    next_arrives = dep_t + travel_time
    hw = next_arrives - expected_dep        
    return hw

In [18]:
control_veh = env.vehicles[17]

rt_id, trip_id = control_veh.route_id, control_veh.next_trips[0].id
min_dep_t, max_dep_t = env.terminal_dep_limits(rt_id, trip_id)
# new min dep t refers to the potential imposed minimum
# based on a layover bus that may be late
pre_hw, new_min_dep_t = env.get_headway(info[1], info[1].iloc[[17]],
                                        min_dep_t, 
                                        terminal=True)

 

In [43]:
next_hw = get_next_headway(env, info[1].iloc[[17]], terminal=True,
                                expected_dep=control_veh.next_trips[0].schedule['departure_sec'].values[0])  

23010.0 0 19438.367860337697


In [35]:
schd = env.routes['91'].schedule.copy()

In [31]:
schd[(schd['stop_sequence']==1) & (schd['stop_id']==14103)].head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,schd_trip_id,direction,block_id,route_id,shape_id,trip_sequence,runid,departure_time_td,departure_sec,confirmed
0,6520011565020,05:12:00,05:12:00,14103,1,Roosevelt,0,0,11565020,South,652000002631,91,65202945,1,F166,0 days 05:12:00,18720.0,1
27,6520022253020,05:25:30,05:25:30,14103,1,Roosevelt,0,0,22253020,South,652000002581,91,65202945,2,F167,0 days 05:25:30,19530.0,1
75,6520011129020,05:39:00,05:39:00,14103,1,Roosevelt,0,0,11129020,South,652000002632,91,65202945,3,F162,0 days 05:39:00,20340.0,1


In [32]:
inf_ = info[0].copy()
inf_[inf_['route_id']=='91'].head(4)

,time,nr_step,route_id,block_id,direction,active,status,status_desc,next_event,next_event_t,t_until_next,stop_id,stop_sequence,pax_load,t_since_last,trip_id,trip_sequence
16,05:23:58.4,150,91,652000002631,South,1,3,active - between stops,arrive at stop,05:23:59.0,0.7,9575,27,0,26.3,11565020,1
17,05:23:58.4,150,91,652000002581,South,0,4,inactive - reported for trip,terminal boarding,05:25:30.0,91.6,14103,1,0,0.0,22253020,2
18,05:23:58.4,150,91,652000002632,South,0,0,inactive - yet to report for first trip,first report at terminal,05:35:23.6,685.2,14103,1,0,NaN,11129020,3
19,05:23:58.4,150,91,652000002541,South,0,0,inactive - yet to report for first trip,first report at terminal,05:54:12.7,1814.4,14103,1,0,NaN,1600020,4


In [ ]:
inf_ = env.info_records[-1].copy()


In [7]:
env, info, n_steps, upd_info = run_di(env, done, n_steps=0, debug_ehd=True)
n_steps, td(env.time)

while loop:   1%|          | 138/12000 [00:02<03:51, 51.26it/s]

----
Time 0 days 05:23:15
Expected: 0 days 05:25:30
Headways: 0 days 00:13:06 -- 0 days 00:13:30
Schedule: 0 days 05:25:30
0 days 05:23:15 < Departure < 0 days 05:30:30
Recommended 0 days 05:25:42 ---> 0 days 05:25:42


(138, Timedelta('0 days 05:23:15'))

In [5]:
env, info, n_steps, control_vehs, rts_info, best_borrow_rt, next_dep_t = run_di(env, done, n_steps=0, debug=True)
n_steps, td(env.time)

while loop:  39%|███▊      | 4637/12000 [01:24<02:14, 54.59it/s]


(4637, Timedelta('0 days 07:06:00'))

In [6]:
next_dep_t

25710.0

In [7]:
best_borrow_rt

'81'

In [8]:
rts_info['81']

,route_id,trip_sequence,runid,block_id,trip_id,shape_id,confirmed,ST_str,ST,AT,DT
0,81,13,F101,652000002734,10006020,65216550,1,06:51:30,24690.0,25202.82636,25218.712592
1,81,14,F116,652000002735,43223020,65216550,1,06:57:30,25050.0,25350.00000,25358.860554
2,81,15,F106,652000002773,43965020,65216550,0,07:03:30,25410.0,NaN,NaN
3,81,16,F114,652000002731,4552020,65216550,0,07:10:30,25830.0,NaN,NaN
4,81,17,F105,652000002763,4712020,65216550,0,07:18:00,26280.0,NaN,NaN
5,81,18,F107,652000002770,4487020,65216550,1,07:25:30,26730.0,NaN,NaN
6,81,19,F108,652000002767,39781020,65216550,1,07:33:30,27210.0,NaN,NaN
7,81,20,F151,652000002685,29097020,65216550,0,07:41:30,27690.0,NaN,NaN
8,81,21,F110,652000002732,27478020,65216550,1,07:49:30,28170.0,NaN,NaN


In [ ]:
def borrow_variables(rt_info, veh_info, env): # veh_info is info[1] from main
    # remove cancelled trips after the current
    base = rt_info[~((rt_info.index>2) & 
                     (rt_info['confirmed']==0))].copy()
    base = base.reset_index(drop=True)
    base['RT']= np.nan

    rt_id = base['route_id'].iloc[0]
    schd = env.routes[rt_id].schedule.copy()
    trip_seq = base['trip_sequence'].iloc[2]
    schd_hw = get_schd_hw(schd, rt_id, trip_seq)

    # without the trip
    actual = base[base['confirmed']==1].reset_index(drop=True)
    for j in range(2, 4):
        veh_block = actual.loc[j, 'block_id']
        veh_df = veh_info[veh_info['block_id']==veh_block].iloc[[0]].copy()
        veh = env.vehicles[veh_df.index[0]]
        report_time = pred_terminal_report_time(
            veh_df, veh, actual.loc[j, 'ST'], 
            env.lines[veh.route_id], env.time) 
        actual.loc[j, 'RT'] = report_time

    sub_cols = ['route_id','trip_sequence', 'ST_str', 'confirmed',
                'ST', 'AT', 'DT', 'RT']
    actual_sub = actual[sub_cols].copy()
    actual_vars = compute_variables(actual_sub)

    # with the trip, index 2 is confirmed==0
    # but make sure index > 3 there are no confirmed == 0
    # and here we only need report time of index=3
    counter = base.copy()
    counter.loc[2, 'confirmed'] = 1
    counter.loc[2,'RT'] = max(env.time, 
                              counter.loc[2, 'ST']-MAX_EARLY_DEV*60) # this is from the current trip!
    counter = counter[counter['confirmed']==1].reset_index(drop=True)

    veh_block = counter.loc[3, 'block_id']
    veh_df = veh_info[veh_info['block_id']==veh_block].iloc[[0]].copy()
    veh = env.vehicles[veh_df.index[0]]
    report_time = pred_terminal_report_time(
        veh_df, veh, counter.loc[3, 'ST'], 
        env.lines[veh.route_id], env.time) 
    counter.loc[3, 'RT'] = report_time

    counter_sub = counter[sub_cols].copy()
    counter_vars = compute_variables(counter_sub)

    dec_vars = write_decision_vars(actual_vars, counter_vars)
    dec_vars['route_id'] = rt_id
    dec_vars['schd_hw'] = schd_hw

    # at what time would it come back to terminal?
    ghost_block = counter.loc[2, 'block_id']
    ghost_trip = counter.loc[2, 'trip_id']
    ghost_arr_t, ghost_schds = get_new_arr_t(rt_id, ghost_block, ghost_trip, env)
    return actual_sub, counter_sub, dec_vars, ghost_arr_t, ghost_schds

In [19]:
actual_dfs = []
counter_dfs = []

lst_dec_vars = []


impact = {}
best_borrow_rt = None
best_bc_ratio = 1.2
next_two_scheds = None
next_dep_t = None

for rt in ['81']:
    rt_info = rts_info[rt].copy()
    act, counter, dec_vars, ghost_arr_t, ghost_schds = borrow_variables(rt_info, info[1], env)
    impact[rt] = change_in_hw(dec_vars)

    actual_dfs.append(act)
    counter_dfs.append(counter)
    lst_dec_vars.append(dec_vars)

    # if next_req_dep_t is None or next_req_dep_t > ghost_arr_t:
    #     benefit = change_in_hw(dec_vars)
    #     cost = impact[lend_rt_id]
    #     ratio = benefit/cost
    #     both_missing = ghost_schds['confirmed'].unique().shape[0] == 1
    #     if ratio > best_bc_ratio and both_missing:
    #         best_borrow_rt = deepcopy(rt)
    #         best_bc_ratio = ratio
    #         next_two_scheds = ghost_schds.copy()
    #         next_dep_t = dec_vars.loc[dec_vars['var']=='next departure', 'counter'].iloc[0]

In [20]:
act

,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT
0,81,13,06:51:30,1,24690.0,24834.998796,24850.286515,NaN
1,81,14,06:57:30,1,25050.0,25350.000000,25367.935108,NaN
2,81,18,07:25:30,1,26730.0,NaN,NaN,26439.983728
3,81,19,07:33:30,1,27210.0,NaN,NaN,27047.114110
4,81,21,07:49:30,1,28170.0,NaN,NaN,NaN


In [21]:
counter

,route_id,trip_sequence,ST_str,confirmed,ST,AT,DT,RT
0,81,13,06:51:30,1,24690.0,24834.998796,24850.286515,NaN
1,81,14,06:57:30,1,25050.0,25350.000000,25367.935108,NaN
2,81,15,07:03:30,1,25410.0,NaN,NaN,25570.294633
3,81,18,07:25:30,1,26730.0,NaN,NaN,26439.983728
4,81,19,07:33:30,1,27210.0,NaN,NaN,NaN
5,81,21,07:49:30,1,28170.0,NaN,NaN,NaN


In [23]:
dec_vars

,var,actual,counter,route_id,schd_hw
0,next departure,26439.983728,26048.967554,81,360.0
1,prior headway,499.713485,499.713485,81,360.0
2,headway,1072.048620,681.032446,81,360.0
3,next headway,770.016272,681.032446,81,360.0


In [ ]:
env, info, n_steps, control_vehs, rts_info, best_borrow_rt, next_dep_t = run_di(env, done, n_steps=n_steps, debug=True)
n_steps, td(env.time)

In [11]:
info = env._display_info()
env.time

23470.706010524835

In [10]:
info[0].head(2)

,time,nr_step,route_id,block_id,direction,active,status,status_desc,next_event,next_event_t,t_until_next,stop_id,stop_sequence,pax_load,t_since_last,trip_id,trip_sequence
1,06:31:10.7,2021,81,652000002734,West,1,3,active - between stops,arrive at stop,06:31:15.3,4.6,3805,32,3,19.4,19931020,5
20,06:31:10.7,2021,91,652000002632,North,1,3,active - between stops,arrive at stop,06:31:17.1,6.4,9634,17,6,14.6,1097020,6


In [12]:
control_vehs = flag_departure_event(info[1])
control_veh = env.vehicles[control_vehs.index[0]]
control_vehs

In [15]:
rts_info = {}
for r in ROUTES:
    terminal_id = OUTBOUND_TERMINALS[r][0]
    # rt_infos[r] = env.routes[r].get_route_info(terminal_id)
    rts_info[r] = env.routes[r].get_status(terminal_id,
                                            recent_trips=2,
                                            max_trips=7)
can_interline = False
borrow_rts = []
for rt in rts_info:
    if rts_info[rt] is None:
        continue
    if rt == control_veh.route_id:
        if not can_lender_interline(rts_info[rt]):
            can_interline = False
            borrow_rts = []
            break # hopefully break just this internal loop
        continue
    rt_info = rts_info[rt].copy()
    if not check_for_interlining(rt_info, env.time, MAX_LATE_DEV*60):
        continue
    # interlining_opps += 1
    can_interline = True
    borrow_rts.append(rt)

In [29]:
borrow_rts

['81']

In [30]:
rts_info['81']

,route_id,trip_sequence,runid,block_id,trip_id,shape_id,confirmed,ST_str,ST,AT,DT
0,81,8,F102,652000002775,44801020,65216550,1,06:15:00,22500.0,22560.480808,22569.545021
1,81,9,F103,652000002772,20296020,65216550,1,06:23:30,23010.0,23239.772511,23250.289144
2,81,10,F112,652000002766,3320020,65216550,0,06:31:00,23460.0,NaN,NaN
3,81,11,F109,652000002769,7518020,65216550,1,06:38:30,23910.0,NaN,NaN
4,81,12,F113,652000002765,24499020,65216550,1,06:45:30,24330.0,NaN,NaN
5,81,13,F101,652000002734,10006020,65216550,0,06:51:30,24690.0,NaN,NaN
6,81,14,F116,652000002735,43223020,65216550,1,06:57:30,25050.0,NaN,NaN


In [31]:
rts_info['91']

,route_id,trip_sequence,runid,block_id,trip_id,shape_id,confirmed,ST_str,ST,AT,DT
0,91,6,F168,652000002561,24452020,65202945,1,06:13:30,22410.0,22410.000000,22428.795893
1,91,7,F111,652000002595,28414020,65202945,1,06:22:30,22950.0,22974.397946,22989.847815
2,91,8,F161,652000002545,16130020,65202945,1,06:32:00,23520.0,NaN,NaN
3,91,9,F166,652000002631,39674020,65202945,0,06:41:30,24090.0,NaN,NaN
4,91,10,F167,652000002581,38441020,65202945,0,07:01:00,25260.0,NaN,NaN
5,91,11,F162,652000002632,48610020,65202945,1,07:11:00,25860.0,NaN,NaN
6,91,12,F661,652000002534,8183020,65202945,0,07:21:30,26490.0,NaN,NaN


In [24]:
td(env.time)

Timedelta('0 days 06:31:11')

In [32]:
rt_info = rts_info['91'].copy()

In [33]:
base = rt_info[rt_info['confirmed']==1].copy()
base = base.reset_index(drop=True)
base['RT'] = np.nan

rt_id = base['route_id'].iloc[0]
schd = env.routes[rt_id].schedule.copy()
trip_seq = base['trip_sequence'].iloc[2]
schd_hw = get_schd_hw(schd, rt_id, trip_seq)

actual = base.copy()
actual.loc[2,'RT'] = env.time

veh_block = actual.loc[3, 'block_id']
veh_df = veh_info[veh_info['block_id']==veh_block].iloc[[0]].copy()
veh = env.vehicles[veh_df.index[0]]
report_time = pred_terminal_report_time(
    veh_df, veh, actual.loc[3, 'ST'], 
    env.lines[veh.route_id], env.time) 
actual.loc[3, 'RT'] = report_time

sub_cols = ['route_id','trip_sequence', 'ST_str', 'confirmed',
            'ST', 'AT', 'DT', 'RT']
actual_sub = actual[sub_cols].copy()
actual_vars = compute_variables(actual_sub)

counter = base.copy()
counter.loc[2, 'confirmed'] = 0
counter = counter[counter['confirmed']==1].copy()
counter = counter.reset_index(drop=True)

NameError: name 'veh_info' is not defined

In [39]:
def lend_variables(rt_info, veh_info, env, control_vehs):
    base = rt_info[rt_info['confirmed']==1].copy()
    base = base.reset_index(drop=True)
    base['RT'] = np.nan
    print(base)

    rt_id = base['route_id'].iloc[0]
    schd = env.routes[rt_id].schedule.copy()
    trip_seq = base['trip_sequence'].iloc[2]
    schd_hw = get_schd_hw(schd, rt_id, trip_seq)

    actual = base.copy()
    actual.loc[2,'RT'] = env.time

    veh_block = actual.loc[3, 'block_id']
    veh_df = veh_info[veh_info['block_id']==veh_block].iloc[[0]].copy()
    veh = env.vehicles[veh_df.index[0]]
    report_time = pred_terminal_report_time(
        veh_df, veh, actual.loc[3, 'ST'], 
        env.lines[veh.route_id], env.time) 
    actual.loc[3, 'RT'] = report_time

    sub_cols = ['route_id','trip_sequence', 'ST_str', 'confirmed',
                'ST', 'AT', 'DT', 'RT']
    actual_sub = actual[sub_cols].copy()
    actual_vars = compute_variables(actual_sub)

    counter = base.copy()
    counter.loc[2, 'confirmed'] = 0
    counter = counter[counter['confirmed']==1].copy()
    counter = counter.reset_index(drop=True)
    
    for j in range(2, 4):
        veh_block = counter.loc[j, 'block_id']
        veh_df = veh_info[veh_info['block_id']==veh_block].iloc[[0]].copy()
        veh = env.vehicles[veh_df.index[0]]
        report_time = pred_terminal_report_time(
            veh_df, veh, counter.loc[j, 'ST'], 
            env.lines[veh.route_id], env.time) 
        counter.loc[j, 'RT'] = report_time
    
    counter_sub = counter[sub_cols].copy()
    counter_vars = compute_variables(counter_sub)

    dec_vars = write_decision_vars(actual_vars, counter_vars)
    dec_vars['route_id'] = rt_id
    dec_vars['schd_hw'] = schd_hw

    veh = env.vehicles[control_vehs.index[0]]
    next_req_dep_t = None

    next_trip_ids = [t.id for t in veh.next_trips[:min(2, len(veh.next_trips))]] #TO-DO: revise?
    if len(veh.next_trips) > 2:
        t_schd = veh.next_trips[2].schedule.copy()
        next_req_dep_t = t_schd['departure_sec'].iloc[0]
    return actual_sub, counter_sub, dec_vars, next_req_dep_t, next_trip_ids

In [40]:
lend_rt_id = control_vehs['route_id'].iloc[0]
actual_dfs = []
counter_dfs = []

lst_dec_vars = []

act, counter, dec_vars, next_req_dep_t, orig_trip_ids = lend_variables(rts_info[lend_rt_id], 
                                                                info[1], env, control_vehs)

  route_id  trip_sequence runid      block_id   trip_id  shape_id  confirmed  \
0       91              6  F168  652000002561  24452020  65202945          1   
1       91              7  F111  652000002595  28414020  65202945          1   
2       91              8  F161  652000002545  16130020  65202945          1   
3       91             11  F162  652000002632  48610020  65202945          1   

     ST_str       ST            AT            DT  RT  
0  06:13:30  22410.0  22410.000000  22428.795893 NaN  
1  06:22:30  22950.0  22974.397946  22989.847815 NaN  
2  06:32:00  23520.0           NaN           NaN NaN  
3  07:11:00  25860.0           NaN           NaN NaN  


KeyError: 3

In [28]:
get_interlining_opp(info, control_vehs, rts_info, borrow_rts, env)

KeyError: 3

In [29]:
control_vehs

,time,nr_step,route_id,block_id,direction,active,status,status_desc,next_event,next_event_t,t_until_next,stop_id,stop_sequence,pax_load,t_since_last,trip_id,trip_sequence
10,0 days 06:41:35.483717233,2845,81,652000002765,East,0,4,inactive - reported for trip,terminal boarding,0 days 06:45:30,0 days 00:03:54.516282767,14102,1,0,0 days,24499020,12


In [31]:
control_veh = env.vehicles[control_vehs.index[0]]

rt_id, trip_id = control_veh.route_id, control_veh.next_trips[0].id
min_dep_t, max_dep_t = env.terminal_dep_limits(rt_id, trip_id)
# new min dep t refers to the potential imposed minimum
# based on a layover bus that may be late

TEST EHD ---------

In [3]:
env, info, n_steps, upd_info = run_ehd(env, done, n_steps=0, debug=True)
n_steps

----
Time 0 days 05:37:29
Expected: 0 days 05:39:00
Headways: 0 days 00:06:23 -- 0 days 00:18:00
Schedule: 0 days 05:39:00
0 days 05:37:29 < Departure < 0 days 05:44:00
Recommended 0 days 05:44:49 ---> 0 days 05:44:00


362

In [29]:
env, info, n_steps, upd_info = run_ehd(env, done, n_steps=n_steps, debug=True)
n_steps

----
Time 0 days 05:55:48
Expected: 0 days 05:56:30
Headways: 0 days 00:08:32 -- 0 days 00:09:00
Schedule: 0 days 05:56:30
0 days 05:55:48 < Departure < 0 days 06:01:30
Recommended 0 days 05:56:44 ---> 0 days 05:56:44


816

In [ ]:
veh_info = info[0].copy()
veh_info[(veh_info['active']==1) | (veh_info['status'].isin([1,2,4]))]

In [ ]:
schd = env.routes['81'].schedule.copy()
schd[(schd['stop_sequence']==1) & (schd['stop_id']==14102)].head(10)

In [4]:
stops = pd.read_csv('data/sim_in/stops.csv', dtype={'route_id':str})

In [31]:
envinfo = env.info_records[-1].copy()

In [32]:
envinfo = envinfo[envinfo['active']==1]

In [ ]:
plot_situation(envinfo, stops, env.hist_date)